In [48]:
%load_ext autoreload
%autoreload 2

In [38]:
from dataset import SyntheticDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from training_data import load_training_data
import torch
from tqdm import tqdm

dataset_path = (
    r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\datasets"
    r"\synthetic\synth_dataset_large.pkl"
    )

sampling_rate = 1.0

X, y = load_training_data(
        dataset_path, column_x='Squiggle', column_y='Motifs',
        sampling_rate=sampling_rate)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=1)

Index(['Squiggle', 'Bases', 'read_id', 'Motifs'], dtype='object')


In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.nn import CTCLoss
from sklearn.preprocessing import normalize

[autoreload of nn failed: Traceback (most recent call last):
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\Parv\anaconda3\envs\pytorch_gpu\Lib\site-packages\IPython\extensions\autoreload.py", line

In [40]:
df = pd.read_pickle(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\data\synthetic\pickled_datasets\25_2_25.pkl")

In [41]:
X = df['squiggle']
Y = df['motif_seq']

In [42]:
ctc = CTCLoss(blank=0)

In [113]:
import torch.optim as optim
from nn import MotifCaller
model = MotifCaller(n_classes=16)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [117]:
ind = 0
for x, y in zip(X, Y):
    len_x = len(x)
    x = torch.tensor(x, dtype=torch.float32).view(1, len_x)
    x = normalize(x)
    print(x.shape)
    x = torch.tensor(x, dtype=torch.float32).view(1, 1, len_x)
    y = torch.tensor(y)
    model_output = model(x)
    #print(model_output.shape)
    #sum_labels = model_output.sum(axis=1)[0].detach().numpy()
    #print(np.exp(sum_labels))
    
    n_timesteps = model_output.shape[1]
    input_lengths = torch.tensor(n_timesteps)
    label_lengths = torch.tensor(len(y))

    loss = ctc(model_output[0], y, input_lengths, label_lengths)
    print(loss.item())
    #print(loss)
    loss.backward()
    optimizer.step()
    ind += 1
    
    #print([np.exp(i) for i in ]])
    if ind == 10:
        break

(1, 8034)
13.123896598815918
(1, 3415)
1.0524003505706787
(1, 9337)
3.56681227684021
(1, 7345)
5.185147285461426
(1, 7380)
6.144494533538818
(1, 11370)
7.255798816680908
(1, 3125)
3.9470643997192383
(1, 7443)
6.6799211502075195
(1, 5454)
5.731595993041992
(1, 3386)
4.050175666809082


In [101]:
print(target_lengths)

tensor(17)


In [106]:
print(input_lengths)

tensor(50)


In [108]:
print(loss)

tensor(6.3805, grad_fn=<MeanBackward0>)
